In [84]:
import pandas as pd
import requests

In [38]:
df = pd.concat([
    [pd.read_excel('Data/table1.xlsx', skiprows=7),
        (pd.read_excel('Data/table2.xlsx', skiprows=7))
    ])

In [40]:
df.columns

Index(['State FIPS Code', 'County FIPS Code', 'State Name', 'County Name',
       'State FIPS Code.1', 'County FIPS Code.1', 'State Name.1',
       'County Name.1', 'Workers in Commuting Flow', 'Margin of Error'],
      dtype='object')

In [44]:
df.columns = [
    "res_st_code",
    "res_county_code",
    "res_state",
    "res_county",
    "work_st_code",
    "work_county_code",
    "work_state",
    "work_county",
    "workers_in_flow",
    "margin_err"
]

In [58]:
df = df[["res_state", "res_county", "work_state", "work_county", "workers_in_flow", "margin_err"]].dropna()

In [82]:
df[df["res_state"] == "Texas"].sort_values(by="workers_in_flow", ascending=False)

,res_state,res_county,work_state,work_county,workers_in_flow,margin_err
101391,Texas,Harris County,Texas,Harris County,2014104.0,8305.0
100720,Texas,Harris County,Texas,Harris County,2014104.0,8305.0
99608,Texas,Dallas County,Texas,Dallas County,1061878.0,6761.0
98578,Texas,Dallas County,Texas,Dallas County,1061878.0,6761.0
98094,Texas,Bexar County,Texas,Bexar County,868962.0,6256.0
...,...,...,...,...,...,...
99295,Texas,Comanche County,Texas,Travis County,1.0,2.0
97517,Texas,Milam County,Texas,Cameron County,1.0,4.0
97512,Texas,Hockley County,Texas,Cameron County,1.0,4.0
98011,Texas,Bexar County,Georgia,DeKalb County,1.0,5.0


In [148]:
# Replace with your Census API Key
api_key = '1da852c7bb61bed30d4d63146465b5cfe4385e5c'
base_url = 'https://api.census.gov/data/2020/dec/pl'


params = {
    'get': 'NAME',  # You can modify this to get more specific fields
    'for': 'place:15000',  # Austin, Texas has the place code of 15000 (this might vary for other cities)
    'in': 'state:48',  # Texas state code
    'key': api_key
}

host = 'https://api.census.gov/data'
year = '/2019'
dataset_acronym = '/acs/acs1'
g = '?get='
variables = 'NAME,B01001_001E'
location = '&for=state:48'

query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
response = requests.get(query_url)

print(query_url)
if response.status_code == 200:
    data = response.json()
    print(response.text)
else:
    print("Error fetching data")

https://api.census.gov/data/2019/acs/acs1?get=NAME,B01001_001E&for=state:48&key=1da852c7bb61bed30d4d63146465b5cfe4385e5c
[["NAME","B01001_001E","state"],
["Texas","28995881","48"]]


In [150]:
def get_variable_table_df(year):
    variable_table_url = f'https://api.census.gov/data/{year}/acs/acs1/variables.html'
    v_table = pd.read_html(variable_table_url)
    variable_df = pd.DataFrame(v_table[0])
    variable_df['Label'].replace({"!!": " ", ":": ""}, regex=True, inplace=True)

    return variable_df

def get_variable_names(variable_table, indeces):
    total_male_by_age_variables = ",".join(variable_table.iloc[indeces[0]: indeces[1]]['Name'].values)
    return total_male_by_age_variables

def get_query_url(year, variables):
    # API Reference: https://www.census.gov/data/developers/guidance/api-user-guide.Example_API_Queries.html
    # Data Dictionary: https://api.census.gov/data.html
    host = 'https://api.census.gov/data'
    year = f'/{year}'
    dataset_acronym = '/acs/acs1'
    g = '?get='
    location = '&for=us:*'
    usr_key = f"&key={USCensusAPI.api_key}"

    query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"

    return query_url

In [181]:
class CommuterDataAPICall():
    def __init__(self):
        self.host = "http://api.census.gov/data/2000/dec/slds"

    def get_query_url(year, variables):
        g = '?get='
        location = '&for=CONCIT:15000'
        usr_key = f"&key=1da852c7bb61bed30d4d63146465b5cfe4385e5c"
        return f"{host}{g}{variables}{location}{usr_key}"
        
    def get_variables(self):
        url = self.host + "/groups/PCT065I.html"
        v_table = pd.read_html(url)
        variable_df = pd.DataFrame(v_table[0])
        variable_df['Label'].replace({"!!": " ", ":": ""}, regex=True, inplace=True)
        return variable_df

CommuterDataAPICall().get_variables()[["Name", "Label"]]

/var/folders/3m/jr7y5tvj0qld31bvfzxkt8440000gn/T/ipykernel_96982/2698856850.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  variable_df['Label'].replace({"!!": " ", ":": ""}, regex=True, inplace=True)


,Name,Label
0,GEO_ID,Geography
1,NAME,Geographic Area Name
2,PCT065I001,Total
3,PCT065I002,"Total Car, truck, or van"
4,PCT065I003,"Total Car, truck, or van Drove alone"
5,PCT065I004,"Total Car, truck, or van Carpooled"
6,PCT065I005,Total Public transportation
7,PCT065I006,Total Public transportation Bus or trolley bus
8,PCT065I007,Total Public transportation Streetcar or troll...
9,PCT065I008,Total Public transportation Subway or elevated
